In [1]:
%matplotlib inline

import glob
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.style.use('ggplot')

In [11]:
## handle bottom-up data
#csvs1 = glob.glob('./bottomup/*csv')
#data1 = {}
#for f in csvs1:
#    content = open(f).read().splitlines()[1:]
#    for line in content:
#        tmp = line.split(',', 1)[0]
#        if tmp not in data1:
#            data1[tmp] = []
#        data1[tmp].append('%s,%s' % (os.path.basename(f).replace('.csv', ''), line))
#for k, v in data1.iteritems():
#    with open('./stock_data/bu_%s.csv' % k, 'wb') as fout:
#        fout.write('\n'.join(v))

# handle top-down data
#csvs2 = glob.glob('./topdown/*csv')
#data2 = {}
#for f in csvs2:
#    content = open(f).read().splitlines()[1:]
#    for line in content:
#        d, v = line.split(',', 1)
#        d = d.replace('-', '')
#        if d not in data2:
#            data2[d] = []
#        data2[d].append(v)
#with open('./stock_data/td.csv', 'wb') as fout:
#    for k, v in data2.iteritems():
#        fout.write('%s,%s\n' % (k, ','.join(v)))

In [25]:
# load top-down data
data = np.loadtxt('./stocks_data/td.csv', delimiter=',')
tdDate = data[:, 0].astype(np.float64).reshape(-1, 1)
tdMat = data[:, 1:]
data = None
print tdMat.shape
print tdMat[:5]

(249, 14)
[[  2.89254787e+00   7.86000000e+01   1.50600000e+02   2.90000000e+00
    6.75000000e+00   4.10000000e+00   7.91133106e-03  -1.50000000e+00
    1.53533586e-01   4.93000000e+01   1.55988054e-01   6.26000000e+01
    5.50000000e+00   3.86900042e-02]
 [  2.89254787e+00   7.86000000e+01   1.50600000e+02   2.90000000e+00
    6.75000000e+00   4.10000000e+00   7.91133106e-03  -1.70000000e+00
    1.53533586e-01   4.91000000e+01   1.55988054e-01   6.26000000e+01
    5.50000000e+00   3.86900042e-02]
 [  2.80684783e+00   8.37000000e+01   1.51700000e+02   2.80000000e+00
    6.09000000e+00   4.10000000e+00  -1.76810316e-02  -1.40000000e+00
    3.57721839e-01   5.36000000e+01   1.86497195e-01   5.25000000e+01
    5.50000000e+00   4.60760885e-02]
 [  2.79063139e+00   8.91000000e+01   1.57100000e+02   3.00000000e+00
    5.63000000e+00   4.10000000e+00  -2.44312890e-02  -1.60000000e+00
    3.09233029e-01   4.97000000e+01   2.01099025e-01   5.93000000e+01
    5.50000000e+00   4.62313910e-02]
 [

In [26]:
# load individual stock data
sid = '00164V10'
data = np.loadtxt('./stocks_data/bu_%s.csv' % sid, delimiter=',', dtype=str)
stockDate = data[:, 0].astype(np.float64)
X = data[:, 3:-1].astype(np.float64)
Y = data[:, 2].astype(np.float64).reshape(-1, 1)
print data.shape
print data[:5]
print X.shape
print X[:5]
print Y.shape
print Y[:5]

(68, 482)
[['20110731' '00164V10' '0.0349897907541' ..., '0.0' '0.0' '2011-07-31']
 ['20110831' '00164V10' '-0.0149891992693' ..., '0.0' '0.0' '2011-08-31']
 ['20110930' '00164V10' '-0.152797324281' ..., '0.0' '0.0' '2011-09-30']
 ['20111031' '00164V10' '0.125949179814' ..., '0.0' '0.0' '2011-10-31']
 ['20111130' '00164V10' '0.0286015755692' ..., '0.0' '0.0' '2011-11-30']]
(68, 478)
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          1.75649261  0.         ...,  0.          0.          0.        ]
 [ 0.          1.78779717  0.         ...,  0.          0.          0.        ]
 [ 0.          1.74342548  0.         ...,  0.          0.          0.        ]
 [ 0.          1.82444104  0.         ...,  0.          0.          0.        ]]
(68, 1)
[[ 0.03498979]
 [-0.0149892 ]
 [-0.15279732]
 [ 0.12594918]
 [ 0.02860158]]


In [28]:
# combine top-down data into individual stock data as new factors
sel = np.in1d(tdDate, stockDate)
X_extended = np.hstack((X, tdMat[sel]))
print X_extended.shape
print X_extended[:5]

(68, 492)
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   2.60985000e+02
    9.03000000e+00  -1.18406414e-01]
 [  0.00000000e+00   1.75649261e+00   0.00000000e+00 ...,   2.63902000e+02
    9.03000000e+00  -1.00000575e-01]
 [  0.00000000e+00   1.78779717e+00   0.00000000e+00 ...,   1.96181000e+02
    9.07000000e+00   1.90464813e-02]
 [  0.00000000e+00   1.74342548e+00   0.00000000e+00 ...,   2.17031000e+02
    9.07000000e+00  -2.65718463e-02]
 [  0.00000000e+00   1.82444104e+00   0.00000000e+00 ...,   2.10333000e+02
    9.07000000e+00  -1.14165179e-02]]


In [31]:
sel = np.in1d(tdDate, [20110731])
print tdMat[sel]

[[  2.95146763e+00   2.03840000e+02   2.74000000e+02   3.20000000e+00
    7.00000000e-02   1.70000000e+00   2.13065719e-01  -8.80000000e+00
    4.13722793e-01   5.91000000e+01   2.06146395e-01   2.60985000e+02
    9.03000000e+00  -1.18406414e-01]]
